In [ ]:
from moviepy.editor import VideoFileClip
import whisper
import os

def convert_mp4_to_mp3(mp4_path, mp3_path):
    video_clip = VideoFileClip(mp4_path)
    audio_clip = video_clip.audio
    audio_clip.write_audiofile(mp3_path)
    audio_clip.close()
    video_clip.close()

def format_time(seconds):
    hours = int(seconds // 3600)
    minutes = int((seconds % 3600) // 60)
    secs = int(seconds % 60)
    millis = int((seconds - int(seconds)) * 1000)
    return f"{hours:02d}:{minutes:02d}:{secs:02d},{millis:03d}"

def process_folder(folder_path):
    # Load whisper model once
    model = whisper.load_model("base")
    
    # Process all mp4 files in the folder
    for filename in os.listdir(folder_path):
        if filename.endswith(".mp4"):
            mp4_path = os.path.join(folder_path, filename)
            mp3_path = os.path.join(folder_path, filename.rsplit(".", 1)[0] + ".mp3")
            srt_path = os.path.join(folder_path, filename.rsplit(".", 1)[0] + ".srt")
            
            print(f"Processing {filename}...")
            
            # Convert MP4 to MP3
            convert_mp4_to_mp3(mp4_path, mp3_path)
            
            # Transcribe audio
            result = model.transcribe(mp3_path)
            
            # Write SRT file
            with open(srt_path, "w", encoding="utf-8") as srt_file:
                for i, segment in enumerate(result["segments"], start=1):
                    start_time = format_time(segment["start"])
                    end_time = format_time(segment["end"])
                    
                    srt_file.write(f"{i}\n")
                    srt_file.write(f"{start_time} --> {end_time}\n")
                    srt_file.write(f"{segment['text'].strip()}\n\n")
            
            print(f"Completed processing {filename}")

# Example usage


In [ ]:

process_folder("./med/")